In [11]:
# ============================================================
# EXPERIMENTO 4 — LATENT DIFFUSION MODEL (LDM-lite)
# ============================================================

import os
import numpy as np
import matplotlib.pyplot as plt
import mlflow

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# ============================================================
# 1. Config MLflow
# ============================================================

# AJUSTAR URL si cambió tu túnel de ngrok
MLFLOW_URL = "https://fleecier-rufus-decadently.ngrok-free.dev"  
mlflow.set_tracking_uri(MLFLOW_URL)
mlflow.set_experiment("Exp4-LatentDiffusion")

# ============================================================
# 2. CARGAR DATASET
# ============================================================

npz_path = "./data/intermediate/pixel_art_data.npz"   # AJUSTAR RUTA SI ES NECESARIO
data = np.load(npz_path)

images = data["images"]       # (N,16,16,3)
# Asegurar normalización correcta (0 a 1) para el Autoencoder
if images.max() > 1.0:
    images = images.astype("float32") / 255.0

print(f"Dataset cargado: {images.shape}, Range: [{images.min()}, {images.max()}]")
images_tf = tf.convert_to_tensor(images, dtype=tf.float32)

# ============================================================
# 3. CARGAR AUTOENCODER (Del Exp 1)
# ============================================================

# Ruta al modelo guardado en el Experimento 1
ae_path = r"D:\documentos\unal\diplomados\diplomado_ml_ds\mod6_metodologias_agiles_desarrollo_proyectos_ml\proyecto\entrega_4_aux\artifacts_exp1\autoencoder_model.keras"

print(f"Cargando Autoencoder desde: {ae_path} ...")
autoencoder = tf.keras.models.load_model(ae_path, compile=False)
autoencoder.trainable = False  # Congelamos el AE, no se entrena aquí

# ------------------------------------------------------------
# 3.1. Extracción Coherente del Encoder (Lógica Exp 1)
# ------------------------------------------------------------
# En Exp 1 usaste: autoencoder.layers[-7].output para los latentes.
# Usamos exactamente la misma lógica para consistencia.

latent_layer_idx = -7
latent_layer = autoencoder.layers[latent_layer_idx]
latent_dim = latent_layer.output.shape[-1]

print(f"Detectado Latent Layer: '{latent_layer.name}' en índice {latent_layer_idx}")
print(f"Dimensión Latente: {latent_dim}")

encoder = models.Model(
    inputs=autoencoder.input,
    outputs=latent_layer.output,
    name="encoder_extracted"
)

# ------------------------------------------------------------
# 3.2. Reconstrucción del Decoder
# ------------------------------------------------------------
# El decoder son todas las capas DESPUÉS de la capa latente (-7).
# Iteramos desde el índice siguiente hasta el final.

decoder_input = layers.Input(shape=(latent_dim,), name="decoder_input_z")
x = decoder_input

# Índice real en la lista de capas (ej. si hay 20 capas, -7 es la 13, empezamos en la 14)
start_idx = len(autoencoder.layers) + latent_layer_idx + 1

for layer in autoencoder.layers[start_idx:]:
    # Ojo: si hay capas de Reshape, hay que tener cuidado. 
    # Generalmente los AE simples funcionan bien así.
    x = layer(x)

decoder = models.Model(inputs=decoder_input, outputs=x, name="decoder_extracted")

print(f"\n✓ Encoder: {encoder.input_shape} -> {encoder.output_shape}")
print(f"✓ Decoder: {decoder.input_shape} -> {decoder.output_shape}")

# Validación rápida
test_z = encoder.predict(images[:1], verbose=0)
test_rec = decoder.predict(test_z, verbose=0)
print(f"Check reconstrucción shape: {test_rec.shape}")


# ============================================================
# 4. PREPROCESAMIENTO LATENTE (CRÍTICO PARA DIFUSIÓN)
# ============================================================

print("\nGenerando representaciones latentes de todo el dataset...")
z_latents = encoder.predict(images, batch_size=512, verbose=1)

# Aplanar si el latente no es plano (aunque layers[-7] suele ser Dense o Flatten)
z_latents = z_latents.reshape((len(z_latents), -1))

# --- ESTANDARIZACIÓN (Z-Score) ---
# Los modelos de difusión asumen ruido N(0,1). 
# Si el AE entrega valores en [0, 10] o [-100, 100], la difusión fallará.
z_mean = np.mean(z_latents, axis=0)
z_std = np.std(z_latents, axis=0) + 1e-6 # Evitar div por cero

z_normalized = (z_latents - z_mean) / z_std

print(f"Latentes normalizados. Mean ~ {np.mean(z_normalized):.2f}, Std ~ {np.std(z_normalized):.2f}")

z_tf = tf.convert_to_tensor(z_normalized, dtype=tf.float32)


# ============================================================
# 5. DDPM SETTINGS (Espacio Latente)
# ============================================================

T = 200                  # Menos pasos que píxeles, el espacio es más simple
LATENT_DIM_FLAT = z_tf.shape[1]
BATCH_SIZE = 128         # Batch más grande porque z pesa poco
EPOCHS = 100             # Se entrena muy rápido, dale más épocas
LR = 1e-3                # Learning rate un poco más agresivo para MLPs

# Schedule de ruido lineal estándar
beta_start = 1e-4
beta_end   = 0.02
betas = np.linspace(beta_start, beta_end, T, dtype=np.float32)

alphas = 1.0 - betas
alphas_cumprod = np.cumprod(alphas, axis=0)

# Constantes para TF
sqrt_alphas_cumprod_tf = tf.constant(np.sqrt(alphas_cumprod))
sqrt_one_minus_alphas_cumprod_tf = tf.constant(np.sqrt(1 - alphas_cumprod))
alphas_tf = tf.constant(alphas)
betas_tf = tf.constant(betas)
alphas_cumprod_tf = tf.constant(alphas_cumprod)


# ============================================================
# 6. FUNCIONES DDPM (Forward & Sampling)
# ============================================================

def extract(a, t, shape):
    out = tf.gather(a, t)
    return tf.reshape(out, (tf.shape(t)[0],) + (1,)*(len(shape)-1))

def q_sample(z0, t, noise):
    """Forward diffusion: añadir ruido al latente z0"""
    return extract(sqrt_alphas_cumprod_tf, t, z0.shape) * z0 + \
           extract(sqrt_one_minus_alphas_cumprod_tf, t, z0.shape) * noise

# ============================================================
# 7. MODELO DE DIFUSIÓN (MLP para Latentes) - CORREGIDO
# ============================================================

def build_latent_denoiser(dim):
    inp_z = layers.Input(shape=(dim,), name="input_z")
    inp_t = layers.Input(shape=(), dtype=tf.int32, name="input_t")

    # ---------------------------------------------------------
    # CORRECCIÓN: Definir la lógica dentro de una función pura
    # ---------------------------------------------------------
    def sinusoidal_embedding_fn(t_tensor):
        embed_dim = 64
        half = embed_dim // 2
        
        # Matemáticas puras de TF
        freqs = tf.exp(tf.range(half, dtype=tf.float32) * -np.log(10000.0) / (half - 1))
        
        # Casting seguro dentro de la función
        t_cast = tf.cast(t_tensor, tf.float32)
        
        # Operaciones vectoriales
        args = t_cast[:, None] * freqs[None, :]
        emb = tf.concat([tf.sin(args), tf.cos(args)], axis=-1)
        
        # Padding si la dimensión es impar
        if embed_dim % 2 == 1:
            emb = tf.pad(emb, [[0, 0], [0, 1]])
            
        return emb

    # ---------------------------------------------------------
    # CORRECCIÓN: Usar Lambda para conectar Keras con TF
    # ---------------------------------------------------------
    t_emb = layers.Lambda(sinusoidal_embedding_fn, output_shape=(64,), name="time_embedding")(inp_t)

    # A partir de aquí, t_emb ya es un tensor procesado compatible con capas Keras
    t_emb = layers.Dense(128, activation="swish")(t_emb)

    # Concatenar z y t
    x = layers.Concatenate()([inp_z, t_emb])
    
    # Red Residual MLP simple
    h1 = layers.Dense(512, activation="swish")(x)
    h2 = layers.Dense(512, activation="swish")(h1)
    h2 = layers.Add()([h1, h2]) # Skip connection
    
    h3 = layers.Dense(512, activation="swish")(h2)
    out = layers.Dense(dim, name="output_noise")(h3)

    return models.Model([inp_z, inp_t], out, name="denoiser_mlp")

# Volver a instanciar y compilar
denoiser = build_latent_denoiser(LATENT_DIM_FLAT)
optimizer = optimizers.Adam(LR)
print("✓ Denoiser construido correctamente.")
print(denoiser.summary())


# ============================================================
# 8. TRAIN LOOP
# ============================================================

dataset = tf.data.Dataset.from_tensor_slices(z_tf)
dataset = dataset.shuffle(10000).batch(BATCH_SIZE)

@tf.function
def train_step(z0):
    bsz = tf.shape(z0)[0]
    t = tf.random.uniform((bsz,), 0, T, dtype=tf.int32)
    noise = tf.random.normal(tf.shape(z0))
    
    # Difundir en espacio latente
    z_t = q_sample(z0, t, noise)

    with tf.GradientTape() as tape:
        # El modelo intenta predecir el ruido añadido
        noise_pred = denoiser([z_t, t], training=True)
        loss = tf.reduce_mean(tf.square(noise - noise_pred))

    grads = tape.gradient(loss, denoiser.trainable_variables)
    optimizer.apply_gradients(zip(grads, denoiser.trainable_variables))
    return loss


# ============================================================
# 9. SAMPLING: Ruido -> Denoise -> Desnormalizar -> Decoder -> Imagen
# ============================================================

def sample_ldm(num_samples):
    # 1. Empezar con ruido gaussiano puro
    z = tf.random.normal((num_samples, LATENT_DIM_FLAT))

    # 2. Iterar hacia atrás T -> 0
    for t in reversed(range(T)):
        t_batch = tf.fill((num_samples,), t)
        
        # Predecir ruido
        eps = denoiser([z, t_batch], training=False)

        # Variables para la fórmula de muestreo
        beta_t = extract(betas_tf, t_batch, z.shape)
        alpha_t = extract(alphas_tf, t_batch, z.shape)
        alpha_bar_t = extract(alphas_cumprod_tf, t_batch, z.shape)
        
        sigma_t = tf.sqrt(beta_t)

        # Ecuación de update (Langevin dynamics / DDPM)
        mean = (1 / tf.sqrt(alpha_t)) * (z - (beta_t / tf.sqrt(1 - alpha_bar_t)) * eps)
        
        if t > 0:
            noise = tf.random.normal(tf.shape(z))
            z = mean + sigma_t * noise
        else:
            z = mean

    # 3. IMPORTANTE: Des-estandarizar z (Volver al rango del Encoder original)
    z_numpy = z.numpy()
    z_denorm = (z_numpy * z_std) + z_mean
    
    # 4. Decodificar a píxeles
    imgs = decoder.predict(z_denorm, verbose=0)
    
    # Clip para asegurar imagen válida
    return np.clip(imgs, 0, 1)


# ============================================================
# 10. EJECUCIÓN CON MLFLOW
# ============================================================

with mlflow.start_run(run_name="Exp4-LatentDiffusion"):

    # Params
    mlflow.log_param("T", T)
    mlflow.log_param("latent_dim", LATENT_DIM_FLAT)
    mlflow.log_param("epochs", EPOCHS)
    mlflow.log_param("batch_size", BATCH_SIZE)
    mlflow.log_param("learning_rate", LR)

    # Entrenar
    for epoch in range(1, EPOCHS+1):
        losses = []
        for batch in dataset:
            loss = train_step(batch)
            losses.append(loss.numpy())

        mean_loss = float(np.mean(losses))
        if epoch % 10 == 0 or epoch == 1:
            print(f"Epoch {epoch}/{EPOCHS} - loss_latent: {mean_loss:.6f}")
        
        mlflow.log_metric("loss", mean_loss, step=epoch)

    # ------------------------------------------
    # 11. Generar Muestras Finales
    # ------------------------------------------
    print("Generando imágenes sintéticas...")
    samples = sample_ldm(64) # Generar 64 imágenes

    # Guardar grid
    fig, axes = plt.subplots(8, 8, figsize=(10, 10))
    k = 0
    for i in range(8):
        for j in range(8):
            axes[i,j].imshow(samples[k])
            axes[i,j].axis("off")
            k += 1
    
    os.makedirs("artifacts_exp4", exist_ok=True)
    grid_path = "artifacts_exp4/ldm_generated_samples.png"
    plt.savefig(grid_path)
    plt.close()
    
    # Log artifacts
    mlflow.log_artifact(grid_path, r"D:\documentos\unal\diplomados\diplomado_ml_ds\mod6_metodologias_agiles_desarrollo_proyectos_ml\proyecto\entrega_4_aux\artifacts_exp4")

    # ------------------------------------------
    # 12. Guardar Modelos y Estadísticas
    # ------------------------------------------
    # Necesitamos guardar denoiser Y las estadísticas (mean/std) para poder usarlo después
    denoiser.save("artifacts_exp4/ldm_denoiser_model.keras")
    
    # Guardar mean/std para inferencia futura
    np.savez("artifacts_exp4/latent_stats.npz", mean=z_mean, std=z_std)
    
    # Log a MLflow
    mlflow.log_artifact("artifacts_exp4/ldm_denoiser_model.keras", r"D:\documentos\unal\diplomados\diplomado_ml_ds\mod6_metodologias_agiles_desarrollo_proyectos_ml\proyecto\entrega_4_aux\artifacts_exp1")
    mlflow.log_artifact("artifacts_exp4/latent_stats.npz", r"D:\documentos\unal\diplomados\diplomado_ml_ds\mod6_metodologias_agiles_desarrollo_proyectos_ml\proyecto\entrega_4_aux\artifacts_exp1")

    print("\n=== EXP4 FINALIZADO EXITOSAMENTE ===")
    print(f"Artifacts guardados en 'artifacts_exp4' y registrados en MLflow.")

Dataset cargado: (89400, 16, 16, 3), Range: [0.0, 1.0]
Cargando Autoencoder desde: D:\documentos\unal\diplomados\diplomado_ml_ds\mod6_metodologias_agiles_desarrollo_proyectos_ml\proyecto\entrega_4_aux\artifacts_exp1\autoencoder_model.keras ...
Detectado Latent Layer: 'flatten' en índice -7
Dimensión Latente: 1024

✓ Encoder: (None, 16, 16, 3) -> (None, 1024)
✓ Decoder: (None, 1024) -> (None, 16, 16, 3)
Check reconstrucción shape: (1, 16, 16, 3)

Generando representaciones latentes de todo el dataset...
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Latentes normalizados. Mean ~ -0.00, Std ~ 0.95
✓ Denoiser construido correctamente.


Model: "denoiser_mlp"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_t             │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_embedding      │ (None, 64)        │          0 │ input_t[0][0]     │
│ (Lambda)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_z             │ (None, 1024)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 128)       │      8,320 │ time_embedding[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 1152)      │          0 │ input_z[0][0],    │
│ (Concatenate)       │                   │            │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 512)       │    590,336 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 512)       │    262,656 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 512)       │          0 │ dense_5[0][0],    │
│                     │                   │            │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 512)       │    262,656 │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_noise        │ (None, 1024)      │    525,312 │ dense_7[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,649,280 (6.29 MB)

 Trainable params: 1,649,280 (6.29 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/100 - loss_latent: 0.793315
Epoch 10/100 - loss_latent: 0.633866
Epoch 20/100 - loss_latent: 0.628379
Epoch 30/100 - loss_latent: 0.626005
Epoch 40/100 - loss_latent: 0.624352
Epoch 50/100 - loss_latent: 0.624283
Epoch 60/100 - loss_latent: 0.623533
Epoch 70/100 - loss_latent: 0.621799
Epoch 80/100 - loss_latent: 0.621833
Epoch 90/100 - loss_latent: 0.621482
Epoch 100/100 - loss_latent: 0.620346
Generando imágenes sintéticas...

=== EXP4 FINALIZADO EXITOSAMENTE ===
Artifacts guardados en 'artifacts_exp4' y registrados en MLflow.
🏃 View run Exp4-LatentDiffusion at: https://fleecier-rufus-decadently.ngrok-free.dev/#/experiments/4/runs/158876ba6a9943379beac92117161f5e
🧪 View experiment at: https://fleecier-rufus-decadently.ngrok-free.dev/#/experiments/4
